# Sentiment Analysis with RNN

In [1]:
# Import and data read

import numpy as np
import tensorflow as tf

with open('reviews.txt', 'r') as f:
  reviews = f.read()
with open('labels.txt', 'r') as f:
  labels = f.read()

In [2]:
reviews[:200]

'bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  '

In [3]:
# Removing punctuation

from string import punctuation

all_text = ''.join([c for c in reviews if c not in punctuation])
reviews = all_text.split('\n')

all_text = ' '.join(reviews)
words = all_text.split()

all_text[:200]

'bromwell high is a cartoon comedy  it ran at the same time as some other programs about school life  such as  teachers   my   years in the teaching profession lead me to believe that bromwell high  s '

In [4]:
# Word examples

words[:5]

['bromwell', 'high', 'is', 'a', 'cartoon']

In [5]:
# Word2int dictionary
vocab_to_int = {k:v for v, k in enumerate(set(words), start = 1)}

# Reviews to integers
reviews_ints = [[vocab_to_int[word] for word in review.split(' ') if word in vocab_to_int.keys()] for review in reviews]

# Labels to integers
labels = [1 if label == 'positive' else 0 for label in labels.split('\n')]

In [6]:
from collections import Counter

review_lens = Counter([len(x) for x in reviews_ints])
print('Zero-length reviews: {}'.format(review_lens[0]))
print('Maximum review length: {}'.format(max(review_lens)))
print('Total reviews {}'.format(len(reviews_ints)))

Zero-length reviews: 1
Maximum review length: 2514
Total reviews 25001


In [7]:
# Remove a review with the length of 0
zero_length_review = [i for i, j in enumerate(reviews_ints) if len(j) == 0]
for i in zero_length_review:
  reviews_ints.pop(i)
  labels.pop(i)

review_lens = Counter([len(x) for x in reviews_ints])
print('Zero-length reviews: {}'.format(review_lens[0]))
print('Maximum review length: {}'.format(max(review_lens)))
print('Total reviews {}'.format(len(reviews_ints)))

Zero-length reviews: 0
Maximum review length: 2514
Total reviews 25000


In [8]:
# Sequence padding (pre)

seq_len = 200

features = []
for review in reviews_ints:
  if len(review) > 200:
    features.append(review[:seq_len])
  else:
    while len(review) < 200:
      review.insert(0, 0)
    features.append(review)
    
print(features[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11729, 44182, 65695, 30458, 23579, 45374, 59461, 63915, 55629, 14774, 64623, 28068, 69945, 65465, 33135, 35441, 52312, 7342, 50771, 19012, 69945, 8039, 35014, 68913, 47299, 14774, 3326, 45514, 70094, 18425, 68379, 6715, 40137, 11729, 44182, 5095, 45747, 65695, 25267, 58493, 68379, 48854, 31721, 65695, 8039, 14774, 9060, 68379, 24179, 38202, 14774, 72551, 9720, 1178, 44866, 54901, 30092, 60443, 51334, 62538, 8039, 26185, 14774, 2744, 13047, 14774, 28337, 5523, 60568, 36033, 18425, 13047, 14774, 68166, 51412, 66585, 53828, 51334, 9720, 37482, 51412, 27915, 14774, 67160, 47299, 59814, 30458, 62326, 44629, 38982, 68379, 14775, 34648, 14774, 7342, 51412, 1352, 4840, 55629, 44182, 30458, 5829, 69734, 69757, 51412, 14286, 36569, 68379, 56937, 69824, 13047, 53899, 8039, 62326, 7452, 68379, 11729, 44182, 51412, 42513

In [22]:
import random

# Train-test split
random.shuffle(reviews_ints)
split_frac = 0.8


train_x, val_x = np.array(features[:int(len(reviews_ints)*split_frac)]), np.array(features[int(len(reviews_ints)*split_frac):])
train_y, val_y = np.array(labels[:int(len(labels)*split_frac)]), np.array(labels[int(len(labels)*split_frac):])

val_x, test_x = np.array(val_x[:int(len(val_x)*0.5)]), np.array(val_x[int(len(val_x)*0.5):])
val_y, test_y = np.array(val_y[:int(len(val_y)*0.5)]), np.array(val_y[int(len(val_y)*0.5):])

print('\t\t\tFeature Shapes:')
print(f'Train set: \t\t{train_x.shape}',
       f'\nValidation set: \t{val_x.shape}',
       f'\nTest set: \t\t{test_x.shape}')

			Feature Shapes:
Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2500, 200)
